In [1]:
from keras import backend as K
from keras.engine import Layer
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout, TimeDistributed
from keras.models import Model

import cv2
import numpy as np

import tensorflow as tf

from IPython.display import display, clear_output

import threading

import pandas as pd

from tqdm import trange, tqdm

Using TensorFlow backend.


In [62]:
input_layer = Input(shape=(400, 300, 3))
nn = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_layer)
nn = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(nn)
nn = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(nn)

nn = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(nn)
nn = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(nn)
nn = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(nn)

nn = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(nn)
nn = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(nn)
nn = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(nn)
nn = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(nn)

nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(nn)
nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(nn)
nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(nn)
nn = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(nn)

nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(nn)
nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(nn)
nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(nn)
# nn = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(nn)

In [46]:
# layer 1
input_layer = Input(shape=(600, 800, 3))
nn = Conv2D(96, (11, 11), strides=(4, 4), name='layer1_conv')(input_layer)

# layer 2
nn = MaxPooling2D((2, 2), strides=(2, 2), name='layer2_pooling')(nn)
nn = Conv2D(256, (5, 5), name='layer2_conv1')(nn)
nn = Conv2D(256, (5, 5), name='layer2_conv2')(nn)
nn = Conv2D(256, (5, 5), name='layer2_conv3')(nn)

In [63]:
nn = Conv2D(512, (3, 3), activation='relu', padding='same', name='rpn_conv1', kernel_initializer='normal')(nn)

classification = Conv2D(9, (1, 1), activation='sigmoid', kernel_initializer='uniform', name='rpn_out_class')(nn)
regression = Conv2D(9 * 4, (1, 1), activation='linear', kernel_initializer='zero', name='rpn_out_regress')(nn)

In [64]:
epsilon = 1e-4

In [65]:
def rpn_loss_cls(num_anchors):
    """Loss function for rpn classification
    Args:
        num_anchors: number of anchors (9 in here)
        y_true[:, :, :, :9]: [0,1,0,0,0,0,0,1,0] means only the second and the eighth box is valid which contains pos or neg anchor => isValid
        y_true[:, :, :, 9:]: [0,1,0,0,0,0,0,0,0] means the second box is pos and eighth box is negative
    Returns:
        lambda * sum((binary_crossentropy(isValid*y_pred,y_true))) / N
    """
    def rpn_loss_cls_fixed_num(y_true, y_pred):

            return 1.0 * K.sum(y_true[:, :, :, :num_anchors] * K.binary_crossentropy(y_pred[:, :, :, :], y_true[:, :, :, num_anchors:])) / K.sum(epsilon + y_true[:, :, :, :num_anchors])
    return rpn_loss_cls_fixed_num

def rpn_loss_regr(num_anchors):
    """Loss function for rpn regression
    Args:
        num_anchors: number of anchors (9 in here)
    Returns:
        Smooth L1 loss function 
                        0.5*x*x (if x_abs < 1)
                        x_abx - 0.5 (otherwise)
    """
    def rpn_loss_regr_fixed_num(y_true, y_pred):

        # x is the difference between true value and predicted vaue
        x = y_true[:, :, :, 4 * num_anchors:] - y_pred

        # absolute value of x
        x_abs = K.abs(x)

        # If x_abs <= 1.0, x_bool = 1
        x_bool = K.cast(K.less_equal(x_abs, 1.0), tf.float32)

        return 1.0 * K.sum(
            y_true[:, :, :, :4 * num_anchors] * (x_bool * (0.5 * x * x) + (1 - x_bool) * (x_abs - 0.5))) / K.sum(epsilon + y_true[:, :, :, :4 * num_anchors])

    return rpn_loss_regr_fixed_num

In [66]:
model = Model(input_layer, [classification, regression])

In [67]:
model.compile(optimizer='adam', loss=[rpn_loss_cls(9), rpn_loss_regr(9)])

In [68]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_inbound_node',
 '_base_init',
 '_built',
 '_check_trainable_weights_consistency',
 '_collected_trainable_weights',
 '_compute_previous_mask',
 '_expects_training_arg',
 '_feed_input_names',
 '_feed_input_shapes',
 '_feed_inputs',
 '_feed_loss_fns',
 '_feed_output_names',
 '_feed_output_shapes',
 '_feed_outputs',
 '_feed_sample_weight_modes',
 '_feed_sample_weights',
 '_feed_targets',
 '_function_kwargs',
 '_get_node_attribute_at_index',
 '_inbound_nodes',
 '_init_graph_network',
 '_init_subclassed_network',
 '_initial_weights',
 '_input_coordinates',
 '_input_layers',
 

In [69]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 400, 300, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 400, 300, 64) 1792        input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 400, 300, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 200, 150, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [8]:
image = cv2.imread('data/FULL_IMAGE_1000x750/RAINY/2016-02-12/camera8/2016-02-12_1509.jpg')
image = np.array([image])
target = np.array([[811,103,869,160]])

In [9]:
image.shape

(1, 750, 1000, 3)

In [10]:
model.fit(x=image, y=[target, target])

Epoch 1/1


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Index out of range using input dim 2; input has only 2 dims
	 [[{{node loss/rpn_out_regress_loss/strided_slice}}]]
	 [[loss/add/_141]]
  (1) Invalid argument: Index out of range using input dim 2; input has only 2 dims
	 [[{{node loss/rpn_out_regress_loss/strided_slice}}]]
0 successful operations.
0 derived errors ignored.

In [0]:
classi, regr = model.predict([image])

In [0]:
regr.shape

In [2]:
anchor_sizes = [16, 32, 64]
anchor_ratios = [[1,1], [1,2], [2,1]]
gxmin, gymin, gxmax, gymax = 811, 103, 869, 160
stride = 16
image_width = 400
image_height = 300

In [3]:
print((int(image_height/stride)+1) * (int(image_width/stride)+1) * len(anchor_ratios) * len(anchor_sizes))

4446


In [4]:
# generate anchor
anchors = []

workers = []
dictionary = {}

def work(identifier, w, h, size, ratio, image_width, image_height):
    a1 = size * ratio[0]
    a2 = size * ratio[1]
    ax1 = w + (a1/2)
    ax2 = w - (a1/2)
    ay1 = h + (a2/2)
    ay2 = h - (a2/2)
    temp = {
        'x' : w,
        'y' : h,
        'xmin' : ax1 if ax1<ax2 else ax2,
        'xmax' : ax1 if ax1>ax2 else ax2,
        'ymin' : ay1 if ay1<ay2 else ay2,
        'ymax' : ay1 if ay1>ay2 else ay2,
        'type' : False
    }
    if temp['xmin'] >= 0 and temp['xmax'] <= image_width and temp['ymin'] >=0 and temp['ymax'] <= image_height:
        dictionary[identifier] = temp
    else:
        dictionary[identifier] = None

ide = 0
for w in trange(0, image_width, stride):
    for h in trange(0, image_height, stride):
        for size in trange(len(anchor_sizes)):
            for ratio in trange(len(anchor_ratios)):
                worker = threading.Thread(target=work, args=(ide, w, h, anchor_sizes[size], anchor_ratios[ratio], image_width, image_height))
                workers.append(worker)
                worker.start()
                ide += 1

for worker in workers:
    worker.join()
    
for i in range(ide):
    if dictionary[i]:
        anchors.append(dictionary[i])
                
print(len(anchors))

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 46.60it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1504.05it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1501.90it/s]

100%|█████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3009.55it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1503.87it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3011.71it/s]

 89%|█████████████████████████████████████████████████████████████████████████▎        | 17/19 [00:00<00:00, 39.84it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|█████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1504.59it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1504.05it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 250.66it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3006.67it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|█████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1503.87it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 273.45it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3011.71it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1522.99it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|█████████████████████

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1504.23it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3005.95it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3007.39it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 250.66it/s]

  0%|                     

...

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1486.11it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2930.35it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2937.19it/s]

100%|█████████████████████

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3006.67it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3008.11it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1486.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 230.88it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                     

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1503.87it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1503.69it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 250.67it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1503.87it/s]


  0%|                     

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3008.11it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 242.62it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1484.88it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1483.13it/s]


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


100%|█████████████████████

3256


In [6]:

for item in anchors:
    image = cv2.imread('data/FULL_IMAGE_1000x750/RAINY/2016-02-12/camera8/2016-02-12_1509.jpg')
    image = cv2.resize(image, (400,300), cv2.INTER_AREA)
#     print(item['xmin'], item['ymin'], item['xmax'], item['ymax'])
    cv2.rectangle(image, (int(item['xmin']), int(item['ymin'])), (int(item['xmax']), int(item['ymax'])), (255,255,255), 2)
    cv2.imshow('image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [6]:
# get data
file = pd.read_csv('annotation_cam8.txt', header=None, names=['filepath', 'xmin', 'ymin', 'xmax', 'ymax', 'label'])
print(file)

                                                filepath  xmin  ymin  xmax  \
0      data/FULL_IMAGE_1000x750/RAINY/2016-02-12/came...   464   601   611   
1      data/FULL_IMAGE_1000x750/RAINY/2016-01-08/came...     0   396   106   
2      data/FULL_IMAGE_1000x750/RAINY/2016-02-12/came...   885    41   935   
3      data/FULL_IMAGE_1000x750/RAINY/2016-01-08/came...   839    39   890   
4      data/FULL_IMAGE_1000x750/OVERCAST/2015-11-16/c...   725   112   791   
5      data/FULL_IMAGE_1000x750/OVERCAST/2015-12-19/c...   119    41   170   
6      data/FULL_IMAGE_1000x750/SUNNY/2016-01-16/came...   567    38   621   
7      data/FULL_IMAGE_1000x750/RAINY/2016-01-08/came...   778    41   828   
8      data/FULL_IMAGE_1000x750/OVERCAST/2015-11-29/c...   283    36   337   
9      data/FULL_IMAGE_1000x750/OVERCAST/2015-11-16/c...   664   396   792   
10     data/FULL_IMAGE_1000x750/OVERCAST/2015-12-18/c...   725   112   791   
11     data/FULL_IMAGE_1000x750/OVERCAST/2015-11-16/c...   101  

In [7]:
file.iloc[0]

filepath    data/FULL_IMAGE_1000x750/RAINY/2016-02-12/came...
xmin                                                      464
ymin                                                      601
xmax                                                      611
ymax                                                      748
label                                                       1
Name: 0, dtype: object

In [9]:
all_image = {}
for i in trange(len(file)):
    data = file.iloc[i]
    filepath = data['filepath']
    xmin = data['xmin']
    ymin = data['ymin']
    xmax = data['xmax']
    ymax = data['ymax']
    label = data['label']
    if filepath not in all_image:
        all_image[filepath] = {
            'gt': []
        }
    temp = {
        'xmin': data['xmin'],
        'ymin': data['ymin'],
        'xmax': data['xmax'],
        'ymax': data['ymax'],
        'label': data['label']
    }
    all_image[filepath]['gt'].append(temp)

100%|██████████████████████████████████████████████████████████████████████████| 24084/24084 [00:04<00:00, 5921.20it/s]


In [10]:
print(all_image)

{'data/FULL_IMAGE_1000x750/RAINY/2016-02-12/camera8/2016-02-12_1509.jpg': {'gt': [{'xmin': 464, 'ymin': 601, 'xmax': 611, 'ymax': 748, 'label': 1}, {'xmin': 392, 'ymin': 38, 'xmax': 446, 'ymax': 92, 'label': 0}, {'xmin': 885, 'ymin': 41, 'xmax': 935, 'ymax': 91, 'label': 1}, {'xmin': 126, 'ymin': 398, 'xmax': 250, 'ymax': 522, 'label': 1}, {'xmin': 334, 'ymin': 185, 'xmax': 417, 'ymax': 269, 'label': 1}, {'xmin': 256, 'ymin': 385, 'xmax': 395, 'ymax': 523, 'label': 1}, {'xmin': 283, 'ymin': 36, 'xmax': 337, 'ymax': 90, 'label': 1}, {'xmin': 726, 'ymin': 37, 'xmax': 780, 'ymax': 91, 'label': 1}, {'xmin': 428, 'ymin': 181, 'xmax': 511, 'ymax': 264, 'label': 1}, {'xmin': 653, 'ymin': 107, 'xmax': 719, 'ymax': 173, 'label': 1}, {'xmin': 512, 'ymin': 181, 'xmax': 595, 'ymax': 264, 'label': 1}, {'xmin': 0, 'ymin': 396, 'xmax': 106, 'ymax': 512, 'label': 1}, {'xmin': 850, 'ymin': 188, 'xmax': 925, 'ymax': 263, 'label': 1}, {'xmin': 156, 'ymin': 109, 'xmax': 221, 'ymax': 174, 'label': 1}, {'xm